In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib import cm

def load_results(filename):
    initial_communities = []
    final_communities = []
    adj_matrix = []
    U = []
    V = []

    with open(filename, 'r') as file:
        lines = file.readlines()

        # Extract adjacency matrix
        adj_matrix_start = lines.index("Adjacency Matrix:\n") + 1
        adj_matrix_end = lines.index("\n", adj_matrix_start)
        for line in lines[adj_matrix_start:adj_matrix_end]:
            adj_matrix.append(list(map(float, line.split())))

        # Extract initial communities (before fuzzy clustering)
        initial_communities_start = lines.index("Initial Communities (Before Fuzzy Clustering):\n") + 1
        initial_communities_end = lines.index("\n", initial_communities_start)
        initial_communities = list(map(int, lines[initial_communities_start:initial_communities_end][0].split()))

        # Extract final communities (after fuzzy clustering)
        final_communities_start = lines.index("Final Communities (After Fuzzy Clustering):\n") + 1
        final_communities_end = lines.index("\n", final_communities_start)
        final_communities = list(map(int, lines[final_communities_start:final_communities_end][0].split()))

        # Extract Membership Matrix (U)
        U_start = lines.index("Membership Matrix (U):\n") + 1
        U_end = lines.index("\n", U_start)
        for line in lines[U_start:U_end]:
            U.append(list(map(float, line.split())))

        # Extract Cluster Centers (V)
        V_start = lines.index("Cluster Centers (V):\n") + 1
        V_end = len(lines)
        for line in lines[V_start:V_end]:
            V.append(list(map(float, line.split())))

    return np.array(adj_matrix), initial_communities, final_communities, np.array(U), np.array(V)

def visualize_graph(adj_matrix, communities=None, title="Graph Visualization"):
    # Create a graph from the adjacency matrix using from_numpy_array
    G = nx.from_numpy_array(adj_matrix)

    # Create a colormap with distinct colors
    num_communities = len(set(communities)) if communities else 1
    cmap = cm.get_cmap('tab20', num_communities)  # 'tab20' provides 20 distinct colors

    # Plot the graph
    plt.figure(figsize=(8, 6))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_size=1000, node_color='lightblue', font_size=15, font_weight='bold')

    # Dictionary to store the community labels and their respective colors
    community_labels = {}

    if communities is not None:
        # Color the nodes based on communities and create the legend
        for i, community in enumerate(communities):
            # Assign each community a color from the colormap
            community_color = cmap(community / num_communities)
            nx.draw_networkx_nodes(G, pos, nodelist=[i], node_color=[community_color], node_size=1000)

            # Store the community label and color for the legend
            community_labels[community] = community_color

    # Add legend outside the plot to avoid overlap with the graph
    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10)
               for color in community_labels.values()]
    labels = [f"Community {community}" for community in community_labels]

    plt.legend(handles=handles, labels=labels, title="Communities", bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)

    plt.title(title)
    plt.tight_layout()  # Adjust layout to ensure the legend doesn't overlap
    plt.show()

# Load the results from the saved file
filename = "fcm_results.txt"
adj_matrix, initial_communities, final_communities, U, V = load_results(filename)

# Visualize the original graph (before fuzzy clustering)
visualize_graph(adj_matrix, communities=initial_communities, title="Graph")

